In [1]:
from nustar_pysolar import planning, io
import astropy.units as u

# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [2]:
fname = io.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2017_150.SHADOW_ANALYSIS.txt


# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [3]:
tlefile = io.download_tle(outdir='../data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

../data/NuSTAR.tle


# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [4]:
tstart = '2017-06-10T12:00:00'
tend = '2017-06-10T20:00:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

# We want to know how to orient NuSTAR for the Sun.

We can more or less pick any angle that we want. But this angle has to be specified a little in advance so that the NuSTAR SOC can plan the "slew in" maneuvers. Below puts DET0 in the top left corner (north-east with respect to RA/Dec coordinates).

### This is what you tell the SOC you want the "Sky PA angle" to be.

In [5]:
pa = planning.get_nustar_roll(tstart, 0)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

NuSTAR Roll angle for Det0 in NE quadrant: 348.3764402915343 deg


# Set up the offset you want to use here:

The first element is the direction +WEST of the center of the Sun, the second is the offset +NORTH of the center of the Sun.

If you want multiple pointing locations you can either specify an array of offsets or do this "by hand" below.

In [6]:
offset = [-190., -47.]*u.arcsec

# Loop over each orbit and correct the pointing for the same heliocentric pointing position.

Note that you may want to update the pointing for solar rotation. That's up to the user to decide and is not done here.

In [15]:
from astropy.coordinates import SkyCoord
for ind, orbit in enumerate(orbits):
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_sky_position(midTime, offset)
    print("Orbit: {}".format(ind))
    print("Orbit start: {} Orbit end: {}".format(orbit[0].isoformat(), orbit[1].isoformat()))
    print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
    print("")
    
    skyfield_pos = planning.get_skyfield_position(midTime, offset, load_path='../data')
    print("Orbit: {}".format(ind))
    print("Orbit start: {} Orbit end: {}".format(orbit[0].isoformat(), orbit[1].isoformat()))
    print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), skyfield_pos[0], skyfield_pos[1]))
    print("")
    
    skyfield_ephem = SkyCoord(skyfield_pos[0], skyfield_pos[1])
    sunpy_ephem = SkyCoord(sky_pos[0], sky_pos[1])
    print(skyfield_ephem.separation(sunpy_ephem))
    
    

Orbit: 0
Orbit start: 2017-06-10T11:25:40 Orbit end: 2017-06-10T12:27:30
Aim time: 2017-06-10T11:56:35 RA (deg): 78.33340314957628 deg Dec (deg): 23.010821208082966 deg

Orbit: 0
Orbit start: 2017-06-10T11:25:40 Orbit end: 2017-06-10T12:27:30
Aim time: 2017-06-10T11:56:35 RA (deg): 78.33946831867041 deg Dec (deg): 23.01126001159926 deg

0d00m20.1592s
Orbit: 1
Orbit start: 2017-06-10T13:02:20 Orbit end: 2017-06-10T14:04:10
Aim time: 2017-06-10T13:33:15 RA (deg): 78.40288022210223 deg Dec (deg): 23.015967821749665 deg

Orbit: 1
Orbit start: 2017-06-10T13:02:20 Orbit end: 2017-06-10T14:04:10
Aim time: 2017-06-10T13:33:15 RA (deg): 78.40894581446256 deg Dec (deg): 23.016403969986666 deg

0d00m20.1591s
Orbit: 2
Orbit start: 2017-06-10T14:39:00 Orbit end: 2017-06-10T15:40:50
Aim time: 2017-06-10T15:09:55 RA (deg): 78.47236160213244 deg Dec (deg): 23.02108406768503 deg

Orbit: 2
Orbit start: 2017-06-10T14:39:00 Orbit end: 2017-06-10T15:40:50
Aim time: 2017-06-10T15:09:55 RA (deg): 78.47842761

# Sanity check: Reproduce the 2016-07-26 pointing.

Based on the IDL version of this code, when the aim time was 2016-07-26 19:53:15.00 the NP was 8.86 degrees and the RA and Dec were 126.04630 and 19.33532 deg, respectively for an offset of +1000 arcseconds West and +150 arcseconds North of the center of the Sun.

In [8]:
aim_time = '2016-07-26T19:53:15.00'
offset = [1000, 150]*u.arcsec
sky_pos = planning.get_sky_position(aim_time, offset)
print(sky_pos)

[ 126.04053869   19.33666449] deg


In [9]:
np = planning.get_nustar_roll(aim_time, 0)
print(np)

8.865884432089102 deg
